In [21]:
import json
import pandas as pd
import networkx as nx
import folium
from scipy.spatial import KDTree

In [22]:
with open('../../../../data/04_final_data/final_edges.geojson', encoding='utf-8') as f:
    edges_geo = json.load(f)
nodes_df = pd.read_csv('../../../../data/04_final_data/final_nodes.csv')
tree_char_df = pd.read_csv('../../../../data/02_intermediate/tree-characteristics.csv')

def get_seasonal_trees(season):
    return set(tree_char_df[tree_char_df[season] == 1]['수목명'])

node_pos = {row['osmid']: (row['lat'], row['lon']) for _, row in nodes_df.iterrows()}

In [23]:
#동대문구 전체 그래프 생성
G = nx.Graph()
for feature in edges_geo['features']:
    props = feature['properties']
    u, v = props['u'], props['v']
    length = props.get('length', 1)
    type_keys = ['park', 'mountain', 'river', 'tree-line', 'road']
    edge_type = next((k for k in type_keys if props.get(k, 0) == 1), 'road') #도로 유형 속성이 1인거 찾기
    tree_names = props.get('tree', '').split() #수목명 리스트로 저장
    G.add_edge(u, v, length=length, type=edge_type, tree=tree_names)

#산책 반경에 맞는 서브그래프 생성
def filter_graph_by_bbox_and_type(G, node_pos, start_lat, start_lon, walk_km, bbox_km=2.0):
    lat_margin = bbox_km / 111.0
    lon_margin = bbox_km / 88.0
    min_lat, max_lat = start_lat - lat_margin, start_lat + lat_margin
    min_lon, max_lon = start_lon - lon_margin, start_lon + lon_margin
    nodes_in_bbox = [osmid for osmid, (lat, lon) in node_pos.items() if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon]
    subG = G.subgraph(nodes_in_bbox).copy()
    return subG

In [24]:
#매력도 계산
def get_preference_score(data, season):
    seasonal_trees = get_seasonal_trees(season)
    type_weight_map = { 'park': 0.3, 'river': 0.2, 'mountain': 0.2, 'tree_line': 0.5, 'road': 2.0}
    type_modifier = type_weight_map.get(data.get('type'), 2.0) 
    tree_modifier = 0.5 if any(t in seasonal_trees for t in data.get('tree', [])) else 1.3
    return type_modifier * tree_modifier

#실제 그래프 엣지 가중치: 매력도 * 거리
def edge_weight(u, v, season, G):
    data = G[u][v]
    preference_score = get_preference_score(data, season)
    return preference_score * data['length']

In [25]:
def recommend_path(G, node_pos, start_lat, start_lon, season, total_walk_km, tolerance=0.2):
    # 시작점 노드 찾기
    node_coords = list(node_pos.values())
    node_osmids = list(node_pos.keys())
    kdtree = KDTree(node_coords)
    _, start_node_idx = kdtree.query([start_lat, start_lon])
    start_osmid = node_osmids[start_node_idx]

    # 서브그래프 생성
    subG = filter_graph_by_bbox_and_type(G, node_pos, start_lat, start_lon, total_walk_km)
    if not subG.has_node(start_osmid):
        print("error: 시작점 주변에 탐색할 경로가 없습니다.")
        return None
        
    # 경로 탐색용 가중치 적용 (매력도+거리)
    for u, v in subG.edges():
        subG[u][v]['weight'] = edge_weight(u, v, season, subG)

    # 매력적인 산책로 찾기
    edge_preferences = [((u, v), get_preference_score(data, season)) for u, v, data in subG.edges(data=True)]
    best_edges = [edge for edge, score in sorted(edge_preferences, key=lambda x: x[1])[:max(1, int(len(edge_preferences) * 0.1))]]
    pois = {n for u, v in best_edges for n in (u, v)}   
    if not pois:
        print("error: 주변에 추천할 만한 경로가 없습니다.")
        return None

    # 시작점에서 POI까지의 최단 경로 길이 계산
    try:
        path_lengths = nx.single_source_dijkstra_path_length(subG, start_osmid, weight='length')
    except nx.NetworkXNoPath:
        print(f"error: 시작점 {start_osmid}이(가) 주변 경로와 연결되어 있지 않습니다.")
        return None

    # 목표 편도 거리에 맞는 POI 후보군 필터링
    candidates = []
    target_one_way_m = total_walk_km * 1000 / 2  # 목표 거리를 절반으로 설정
    for poi in pois:
        if poi in path_lengths: #우리가 설정한 시간만에 갈수있는지
            path_length_to_poi = path_lengths[poi]
            # 편도 거리를 기준으로 비교
            if abs(path_length_to_poi - target_one_way_m) <= target_one_way_m * tolerance:
                candidates.append((poi, path_length_to_poi))

    if not candidates:
        print(f"error: {total_walk_km}km 왕복 거리에 맞는 추천 경로를 찾지 못했습니다.")
        return None

    # 목표 편도 거리에 가장 근접한 산책로를 목적지로 선택
    best_poi, _ = min(candidates, key=lambda x: abs(x[1] - target_one_way_m))
    # 가중치(매력도+거리) 반영해서 최적 경로 계산
    final_path = nx.shortest_path(subG, start_osmid, best_poi, weight='weight')
    
    return final_path

In [26]:
start_lat, start_lon = 37.562151, 127.065117
walk_km = 4  # 총 왕복 산책 거리
season = '여름'

path_nodes = recommend_path(G, node_pos, start_lat, start_lon, season, walk_km)

if path_nodes:
    # 편도 경로 길이 계산
    one_way_length = sum(G.edges[path_nodes[i], path_nodes[i+1]]['length'] for i in range(len(path_nodes)-1))
    
    #결과
    print(f"산책로 추천:")
    print(f"왕복 시 예상 거리: {one_way_length*2/1000:.2f} km")
    round_trip_time_min = one_way_length * 2 / (4000 / 60) #걷는 속도 평균 4km/h = 66.67m/min
    print(f"예상 소요 시간(왕복): {round_trip_time_min:.0f}분")


    #그래프 시각화
    m = folium.Map(location=[start_lat, start_lon], zoom_start=15) #지도 객체 생성
    folium.Marker(location=[start_lat, start_lon], popup="시작점", icon=folium.Icon(color='red')).add_to(m) #시작점 마커
    end_pos = node_pos[path_nodes[-1]]
    folium.Marker(location=end_pos, popup="반환점", icon=folium.Icon(color='blue')).add_to(m) #도착점 마커 

    path_coords = [node_pos[node] for node in path_nodes] #경로 표시
    folium.PolyLine(locations=path_coords, color='purple', weight=5, opacity=0.8).add_to(m)
    
    m.save('recommended_walk_path.html')
    print("\n'recommended_walk_path.html' 파일이 저장되었습니다.")

else:
    print('error: 경로를 찾지 못하여 지도를 생성하지 않았습니다.')

산책로 추천:
왕복 시 예상 거리: 6.75 km
예상 소요 시간(왕복): 101분

'recommended_walk_path.html' 파일이 저장되었습니다.
